# Obtaining structured web content and programmatic access

In this notebook we will learn how to make use of readily structured data through dedicated application programming interfaces (APIs), how to authenticate and how to properly design requests (also called "payload") in order to retrieve large datasets.

**Advantages** of APIs are that
- access is legal and in most cases clearly and transparently regulated (e.g. 10,000 calls per day)
- structuring through `requests` and `BeautifulSoup` not required
- Python packages that simplify server-client interaction are available

**Disadvantages** of APIs are that
- we have to learn how APIs work and how we should interact with them (each API has some peculiarities and documentation is usually good, but sometimes not so...)
- authentification may be required and access may not be free of charge 

We will
- obtain data of a public statistical office such as the IMF or World Bank through the `pandas-datareader`
- directly obtain a (ranking) table from a website such the [World Cube Association](https://www.worldcubeassociation.org/results/rankings/333/single)
- learn how to use the Destatis/GENESIS Online service and API
- learn how to use the Twitter API (in particular [Tweepy](https://www.tweepy.org/), a Python library for the Twitter API) and retrieve Tweets with GeoTags (i.e. coordinates) subject to specified geography and search terms
- conduct some small analyses and visualise the results appropriately

In [ ]:
import pandas as pd

In [ ]:
!pip install pandas-datareader --upgrade

In [ ]:
from pandas_datareader import wb # imports world bank access
search = wb.search('GDP.*current.*US') # search for keyword
search

In [ ]:
df = wb.download(indicator = 'NY.GDP.MKTP.CD', country = ['DE', 'FR', 'IT'],
                start = 2000, end = 2019)
df.info()

In [ ]:
df.groupby('country').describe()

In [ ]:
df2 = df.reset_index()
df2.head()

In [ ]:
df2.info()

In [ ]:
df2.columns = ['country', 'year', 'gdp'] # rename columns
df2.year = df2.year.astype(int) # numeric annual indicator
df2.info()

In [ ]:
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
# set(df2['country'])

In [ ]:
traces = []

for country in df2['country'].unique():
    
    trace = go.Scatter(name = country, x = df2[df2['country'] == country]['year'],
                      y = df2[df2['country'] == country]['gdp'])
    
    traces.append(trace)
    
fig = go.Figure(data = traces)

fig.layout.update(title=go.layout.Title(

    text = 'Nominal GDP'

))

fig.layout.update(yaxis= go.layout.YAxis(title=go.layout.yaxis.Title(
                        text='in US$')));

fig.layout.update(xaxis = go.layout.XAxis(title = go.layout.xaxis.Title(text = 'Year'), rangeslider = dict(visible = True)));

iplot(fig, filename = 'Nominal_GDP')

## Directly parsing `table` objects from HTML

In [ ]:
ranking = pd.read_html('https://www.worldcubeassociation.org/results/rankings/333/single', encoding='utf-8')

In [ ]:
ranking[0].head()

Which nationality appears most frequently in the World Cube Association's ranking?

In [ ]:
ranking[0].groupby('Citizen of').count()

Which nationality needed, on average, the **lowest** amount of time to solve a 3x3x3 cube? Sort the output in ascending order.

In [ ]:
ranking[0].groupby('Citizen of').mean()['Result'].sort_values(ascending = True)

How many possible states/permutations, starting from the solved state, can a 3x3x3 Rubik's cube have? 

Hints:
1. The centre squares are fixed (a plane rotation around these squares doesn't change the cube's state)
2. There are eight corner pieces (with three colors on the side) and twelve edge pieces (with two colors on the side) which all revolve around the centre pieces
3. There are six different colors
4. We only look at "legal" states, i.e. those that can only be realised without assembling the cube (and therefore not violating Hint 1)

In [ ]:
import math

answer = (math.factorial(8) * 3**8) * 1/3 * (math.factorial(12) * 2**12) * 1/2 * 1/2

# [(corner pieces) * fraction of admissible corner combinations (clock-wise and anti-clockwise twists cancel each other out, 
# hence 3**7 / 3**8 = 1/3)] 
# * [(edge pieces) * fraction of admissible edge combinations (clock-wise and anti-clockwise twists cancel each other out, 
# hence 2**11 / 2**12 = 1/2)]
# * [1/2 (only half of the corner and edge states can be reached as corner and edge states must both coincide in the number
# i.e. even or odd of turns taken to reach either position)]

print(str(answer) + " or about 43.2 quintillion combinations!")

Which result entry in the World Ranking table is the most recent one? Which one is the oldest one? Be as precise as possible! (Hint: You may have to combine your knowledge from scraping HTML files.)

Compute the expected value of `Result` conditional on `Nationality = 'Germany'`. Are the German contestants statistically significantly faster/slower in solving the cube than other contestants, based on `Nationality`? Does statistical significance change if you use robust standard errors?

In [ ]:
!pip install statsmodels

In [ ]:
import statsmodels.api as sm

In [ ]:
df = ranking[0]
df['Non_German_Dummy'] = 1

In [ ]:
non_german_index = df[df['Citizen of'] != 'Germany'].index

In [ ]:
german_index = [x for x in df.index if x not in non_german_index]

In [ ]:
dummy_values = [0 if y in german_index else 1 for y in df.index]

In [ ]:
X = dummy_values
Y = df['Result']

X = sm.add_constant(X) # adding a constant
model = sm.OLS(Y, X).fit()
print_model = model.summary()
print(print_model)

In [ ]:
model.get_robustcov_results(cov_type='HC1').summary()

## Spatial libraries

The installation procedure of spatial libraries for Python (on Windows) can be quite tedious but [this answer](https://stackoverflow.com/questions/51095970/install-python-geopandas-failed/51560940#51560940) on Stackoverflow (make sure to upvote ;)) and [this detailed instruction](https://geoffboeing.com/2014/09/using-geopandas-windows/) make it straight forward. You can also find the required wheels for Python 3.8 and 64-bit for offline `pip install` in this notebook's [repository](https://github.com/gerwolf/webscraping-workshop/tree/main/DataFrames%20and%20APIs). After this, you can simply `pip install geopandas`.

In [38]:
import geopandas as gpd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world.plot()

In [ ]:
world.head()

In [ ]:
df = pd.read_csv("https://gist.githubusercontent.com/gerwolf/81ebb170eb25d4a13f2695db4520f90e/raw/7c24eb4a2b11f6d4c7ac38bdf3555dfc7da6823e/GDPpc_PPP.csv", sep = ";")
df['Country'] = df['Country'].str.replace('�',' ')
df['Country'] = df['Country'].str.replace('United States','United States of America')

In [ ]:
df[df['Country'] == 'United States of America']

In [ ]:
final_geodf = df.merge(world, left_on='Country', right_on = 'name', how = 'left').dropna()

In [ ]:
final_geodf['GDPpc_PPP'] = final_geodf['GDPpc_PPP'].str.replace(',','')

In [ ]:
fig = go.Figure(data=[go.Choropleth(
    locations=final_geodf['iso_a3'], # Spatial coordinates
    z = final_geodf['GDPpc_PPP'].astype(float), # Data to be color-coded
#     locationmode = 'world', # set of locations match entries in `locations`
    colorscale = 'Reds',
    text=final_geodf['Country']
#     colorbar_title = "Millions USD",
)])

fig.layout.update(
    title_text = 'GDP per capita (2018, IMF)',
    geo_scope='world', # limite map scope to USA
);

iplot(fig, filename ="geomap")

In [ ]:
fig = go.Figure(data=[go.Choropleth(
    locations=final_geodf[final_geodf['continent'] == 'Europe']['iso_a3'], # Spatial coordinates
    z = final_geodf[final_geodf['continent'] == 'Europe']['GDPpc_PPP'].astype(float), # Data to be color-coded
#     locationmode = 'world', # set of locations match entries in `locations`
    colorscale = 'Reds',
    text=final_geodf[final_geodf['continent'] == 'Europe']['Country']
#     colorbar_title = "Millions USD",
)])

fig.layout.update(
    title_text = 'GDP per capita (2018, IMF)',
    geo_scope='europe', # limite map scope to USA
);

iplot(fig, filename ="geomap")

## Destatis/GENESIS Online
The GENESIS API is the web interface service by the Federal Statistical Office of Germany and is a good place to start learning how to interact programmatically with a server.

There is a [comprehensive description/introduction](https://www-genesis.destatis.de/genesis/misc/GENESIS-Webservices_Einfuehrung.pdf) on the service, unfortunately in German only. To display the PDF inside Jupyter Notebook in Chrome you may have to enable the [PDF Viewer extension](https://chrome.google.com/webstore/detail/pdf-viewer/oemmndcbldboiebfnladdacbdfmadadm?utm_source=chrome-ntp-icon).

In [ ]:
from IPython.display import IFrame, display
filepath = "https://www-genesis.destatis.de/genesis/misc/GENESIS-Webservices_Einfuehrung.pdf"
IFrame(filepath, width=980, height=800)

Read about the `whoami` method (in Section 2.2). Do you have to authenticate? What does it return? How would you send a `request`?

In [ ]:
import genesis_config
import requests

In [ ]:
url = "https://www-genesis.destatis.de/genesisWS/rest/2020/helloworld/whoami"
response = requests.get(url)
response.text

Read about the `logincheck` method (in Section 2.2). Do you have to authenticate? What does it return? Construct a request object using string formatting, send a `request` (in English language) and print the request's status. What type is the response's `text` attribute?

In [ ]:
url = 'https://www-genesis.destatis.de/genesisWS/rest/2020/helloworld/logincheck?username={}&password={}&language={}'.format(genesis_config.Username, genesis_config.Password, 'en')
response = requests.get(url)

In [ ]:
print(response.text)

Now that we have a working connection to the GENESIS Online API we want to directly obtain an economic indicator, the private sector's savings rate on a quarterly basis, for instance. This `data` is usually stored in a `table` somewhere in the depths of a data warehouse and it is (unfortunately) necessary to familiarise yourself, at least partially, with the internal server's structure.
1. In the documentation file search for the `tablefile` method (under Section 2.5 Data). When should it be used? What does it return?
2. Which method should you use if you want to directly obtain a `chart`?
3. Which method should you use if you want to directly obtain a regional `map`? Which parameter controls the image's resolution?

In [ ]:
field = '12411-0010'
stand = '31.12.2019'
language = 'en'
url = 'https://www-genesis.destatis.de/genesisWS/rest/2020/data/map2table?username={}&password={}&name={}&area=all&mapType=0&classes=5&classification=0&zoom=3&startyear=&endyear=&timeslices=&regionalvariable=&regionalkey=&classifyingvariable1=&classifyingkey1=&classifyingvariable2=&classifyingkey2=&classifyingvariable3=&classifyingkey3=&format=png&stand={}&language={}'.format(genesis_config.Username, genesis_config.Password, field, stand, language)
response = requests.get(url)

In [ ]:
with open("map.png", 'wb') as f:
    f.write(response.content)

In [ ]:
from IPython.display import Image
Image(filename='map.png') 

4. Login to the [GENESIS Online user interface](https://www-genesis.destatis.de/genesis/online?Menu=Anmeldung#abreadcrumb). Familiarise yourself with the tables' structure and navigate to the National Accounts (at the central level) --> Private sector disposable income and savings at quarterly frequency. Which parameters in the request can you control?
5. Which method would you choose if you want to directly obtain a `table` in some machine readable format, e.g. a `.csv` or `.xlsx` that you can read into `pandas`? How do you include additional conditions matching particular values?
6. Construct a `request` which contains the following specification:
    - only seasonally and calendar-adjusted values (X13)
    - all available years and quarters
    - output format should be a `.xlsx` file
7. Send the request but directly through the `pandas.read_excel()` method.

In [ ]:
! pip install openpyxl

In [ ]:
code = '81000-0010'
key_1 = 'WERT04'
val_1 = 'X13JDKSB'
key_2 = ''
val_2 = ''
key_3 = ''
val_3 = ''
start_year = '1991'
end_year = '2020'

url = ('https://www-genesis.destatis.de/genesisWS/rest/2020/data/tablefile?username={}&password={}&name={}&area=DINSG*&compress=false&transpose=false&startyear={}&endyear={}&timeslices=&regionalvariable=&regionalkey=&classifyingvariable1={}&classifyingkey1={}&classifyingvariable2={}&classifyingkey2={}&classifyingvariable3={}&classifyingkey3={}&format=xlsx&job=false&stand=&language=de').format(genesis_config.Username, genesis_config.Password, code, start_year, end_year, key_1, val_1, key_2, val_2, key_3, val_3)
df = pd.read_excel(url)

In [ ]:
df

In [ ]:
import numpy as np

In [ ]:
df = df.iloc[[2,3,37], :].T.iloc[2:,:]
df.columns = ['Year', 'Quarter', 'Rate']
df.reset_index(inplace = True, drop = True)
df['Year'] = df['Year'].fillna(method='ffill')
df['Quarter_str'] = df['Quarter'].copy()
df['Quarter'] = df['Quarter'].replace('1. Quartal', 'Q1')
df['Quarter'] = df['Quarter'].replace('2. Quartal', 'Q2')
df['Quarter'] = df['Quarter'].replace('3. Quartal', 'Q3')
df['Quarter'] = df['Quarter'].replace('4. Quartal', 'Q4')
df['Rate'] = df['Rate'].replace ('...', np.NaN)
qs = df['Year'] + '-' + df['Quarter']
df['Date'] = pd.PeriodIndex(qs.values, freq='Q').to_timestamp()
df.set_index(df['Date'], inplace = True, drop = True)
del df['Date']
df.dropna(inplace=True)
df['col_name'] = df['Quarter'] + ' ' + df['Year'].str[2:4]
col_names = list(df['col_name'].values)
df = df.T
df.columns = col_names
df = df.T

In [ ]:
df.head()

In [ ]:
fig = go.Figure(data=[
    
    go.Scatter(name='Private sector savings rate', x = list(df.index),
    y = list(df['Rate']))
    
])

fig.layout.update(title = go.layout.Title(
                        text='Private sector savings rate (Germany)'))

fig.layout.update(yaxis= go.layout.YAxis(title=go.layout.yaxis.Title(
                        text='in % of disposable income')))

fig.layout.update(xaxis = go.layout.XAxis(title = go.layout.xaxis.Title(text = 'Quarter-Year'), rangeslider = dict(visible = True)));

iplot(fig, filename = 'savings_rate')

## Twitter API


In [ ]:
!pip install python-twitter --upgrade

Note there are [rate limits](https://developer.twitter.com/en/docs/twitter-api/rate-limits)!

In [1]:
import twitter
import twitter_config

api = twitter.Api(consumer_key = twitter_config.api_key ,
                  consumer_secret = twitter_config.api_secret_key,
                  access_token_key = twitter_config.access_token,
                  access_token_secret = twitter_config.access_token_secret,
                  tweet_mode = 'extended',
                  sleep_on_rate_limit = True)

In [ ]:
got = api.GetSearch('#gameofthrones',
                       count = 100) 

In [ ]:
example_tweet = got[0]
type(example_tweet)

In [ ]:
print('id:', example_tweet.id)
print('Text:', example_tweet.full_text) # key 'text' wenn tweet_mode != 'extended'
print('Hashtags:', example_tweet.hashtags)
print('Media:', example_tweet.media)
print('Date:', example_tweet.created_at)
print('Language:', example_tweet.lang)
print('Retweets:', example_tweet.retweet_count)

In [ ]:
!pip install tweepy --upgrade

In [24]:
import tweepy
import json
import numpy as np

In [25]:
auth = tweepy.OAuthHandler(twitter_config.api_key, twitter_config.api_secret_key)
auth.set_access_token(twitter_config.access_token, twitter_config.access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [26]:
searchTerms = "#maskenaffaere"
noOfSearch = 200
searchCountry = "Germany"
places = api.geo_search(query=searchCountry, granularity="country")
place_id = places[0].id

In [27]:
place_id

'fdcd221ac44fa326'

In [ ]:
# tweets = tweepy.Cursor(api.search , q='{} place:{}'.format(searchTerms, place_id) and ("place:%s" % place_id), lang="de", tweet_mode = 'extended', count = 100).items(noOfSearch)

In [ ]:
# tweet_list = []

# for tweet in tweets:
    
#     tweet_list.append(tweet)
    
# D = [i._json for i in tweet_list]

# with open('First_bunch.json', 'w', encoding='utf-8') as f:
#     json.dump(D, f, ensure_ascii=False)
    
# tweet_ids_list = []

# for tweet in tweet_list:
    
#     tweet_ids_list.append(tweet._json['id'])
    
# max_id = tweet_ids_list[-1] 

In [ ]:
# max_id

In [29]:
max_id = None
for j in range(50):
    
    header = {
        # "q": f'{searchTerms} place:{place_id}'.format(searchTerms, place_id) and ("place:%s" % place_id),
        # "q": "#maskenaffaere -filter:locations",
        # "q": "#maskenaffaere filter:geo_enabled",
        # "q": "#maskenaffaere",
        # "geocode": '51.590556,10.106111,310mi',
        "q": "place:fdcd221ac44fa326",
        "lang": "de",
        "tweet_mode": "extended",
        "count": 100,
        "max_id": max_id,
        # "search_term": ""
    }
    
    try:
    
        batch_tweets = tweepy.Cursor(api.search, **header).items(noOfSearch)
        
    except Exception as error:
        
        break
        
    batch_tweets = [i._json for i in batch_tweets]
    batch_ids = [i["id"] for i in batch_tweets]
    with open(f'{j}_de_bunch.json', 'w', encoding='utf-8') as f:
        json.dump(batch_tweets, f, ensure_ascii=False)
    max_id = batch_ids[-1]

In [31]:
bunches = []

for bunch in range(50):
    
    with open('./Twitter Bunches/' + str(bunch) + '_de_bunch.json', 'r', encoding='utf-8') as f:
    
        D_read = json.load(f)
        bunches.extend(D_read) # extend instead of append

In [74]:
bunch_no = 1
tweet_url = 'https://twitter.com/{}/status/{}'.format(bunches[bunch_no]['user']['screen_name'], bunches[bunch_no]['id'])

In [75]:
tweet_url

'https://twitter.com/miasanmetz/status/1380969144032636930'

In [76]:
bunches[bunch_no]['place']['bounding_box']['coordinates'][0]

[[11.360589, 48.061634],
 [11.722918, 48.061634],
 [11.722918, 48.248124],
 [11.360589, 48.248124]]

In [44]:
from shapely.geometry import Point, Polygon

In [85]:
# p1 = Point(bbox[0], bbox[3])
# p2 = Point(bbox[2], bbox[3])
# p3 = Point(bbox[2], bbox[1])
# p4 = Point(bbox[0], bbox[1])



p1 = Point(bunches[bunch_no]['place']['bounding_box']['coordinates'][0][0][0], bunches[bunch_no]['place']['bounding_box']['coordinates'][0][3][1])
p2 = Point(bunches[bunch_no]['place']['bounding_box']['coordinates'][0][1][0], bunches[bunch_no]['place']['bounding_box']['coordinates'][0][3][1])
p3 = Point(bunches[bunch_no]['place']['bounding_box']['coordinates'][0][1][0], bunches[bunch_no]['place']['bounding_box']['coordinates'][0][0][1])
p4 = Point(bunches[bunch_no]['place']['bounding_box']['coordinates'][0][0][0], bunches[bunch_no]['place']['bounding_box']['coordinates'][0][0][1])

np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])

bb_polygon = Polygon([np1, np2, np3, np4])

df2 = gpd.GeoDataFrame(gpd.GeoSeries(bb_polygon), columns=['geometry'])

In [86]:
centroid_extract = bb_polygon.centroid

In [87]:
centroid_extract.xy[1][0], centroid_extract.xy[0][0]

(48.154878999999994, 11.5417535)

In [ ]:
df = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])

In [88]:
tweet_ids = []

for tweet in bunches:
    
    tweet_ids.append(tweet['id'])

In [89]:
len(np.unique(np.array(tweet_ids)))

9951

In [90]:
len(bunches)

10000

In [91]:
bunches[-1]

{'created_at': 'Fri Apr 09 23:19:03 +0000 2021',
 'id': 1380661567109337103,
 'id_str': '1380661567109337103',
 'full_text': '@HertlIrene @Erweckungszeit Hast du deine Errettung mit Eifersucht wertgeschätzt? Stattdessen gibt es Gnadenmisbrauch in der Gemeinde!\n#RaptureIsCloserNow',
 'truncated': False,
 'display_text_range': [28, 154],
 'entities': {'hashtags': [{'text': 'RaptureIsCloserNow',
    'indices': [135, 154]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'HertlIrene',
    'name': 'IRENE',
    'id': 1283871223378784256,
    'id_str': '1283871223378784256',
    'indices': [0, 11]},
   {'screen_name': 'Erweckungszeit',
    'name': 'JESUS kommt',
    'id': 890107501781364737,
    'id_str': '890107501781364737',
    'indices': [12, 27]}],
  'urls': []},
 'metadata': {'iso_language_code': 'de', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': 1380634251343839232,
 'in_rep